Md. Khadem Ali
Student, Dept. of Geography & Environment, Shahid M. Mansur Ali College (Affiliated with National University of Bangladesh)
Portfolio: wwww.khademali.com

## পর্ব ৯: Classification of Remote Sensing Data

এই নোটবুকে আমরা শিখব:
1. Classification কী এবং কেন প্রয়োজন
2. Supervised vs Unsupervised Classification
3. Preparing Training Data
4. Supervised Classification উদাহরণ (Random Forest)
5. Accuracy Assessment
6. Visualization of Classified Map

## 1. Classification Introduction

Remote sensing data classification হলো পিক্সেল বা অবজেক্টকে predefined classes (যেমন: water, vegetation, urban) এ ভাগ করা।  
- উদাহরণ: Land Cover Mapping, Urban Expansion Monitoring

## 2. Supervised vs Unsupervised Classification

- Supervised Classification: Training data দেওয়া থাকে, model শিখে class predict করে।  
- Unsupervised Classification: Model নিজে clustering করে, কোন পিক্সেল কোন group এ যাবে তা স্বয়ংক্রিয়ভাবে নির্ধারণ করে।

## 3. Preparing Training Data

Training data তৈরি করতে:
1. Field survey বা reference map থেকে labeled points সংগ্রহ করা
2. Pixel values বা raster values sampling
3. GeoDataFrame বা DataFrame আকারে structured data তৈরি

In [ ]:
import geopandas as gpd
import rasterio
import pandas as pd
import numpy as np

# Labeled points লোড
train_points = gpd.read_file('data/training_points.shp')

# Raster layers লোড (multiple bands)
bands = ['data/band1.tif', 'data/band2.tif', 'data/band3.tif']
rasters = [rasterio.open(b) for b in bands]

# Extract raster values at training points
values = []
for point in train_points.geometry:
    vals = [r.sample([(point.x, point.y)]).__next__()[0] for r in rasters]
    values.append(vals)

# DataFrame
X_train = pd.DataFrame(values, columns=['band1','band2','band3'])
y_train = train_points['class_label']
X_train.head()

## 4. Supervised Classification: Random Forest

Random Forest ব্যবহার করে supervised classification করা।

In [ ]:
from sklearn.ensemble import RandomForestClassifier

# মডেল train
rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X_train, y_train)

# Prediction over full raster (example for small area)
# ধাপে ধাপে raster array তৈরি করা
sample_band = rasters[0].read(1)
rows, cols = sample_band.shape
stacked_rasters = np.dstack([r.read(1) for r in rasters])
flat_pixels = stacked_rasters.reshape(-1, stacked_rasters.shape[2])

# Prediction
pred_flat = rf.predict(flat_pixels)
pred_map = pred_flat.reshape(rows, cols)

## 5. Accuracy Assessment

Confusion matrix, overall accuracy, kappa coefficient দিয়ে accuracy যাচাই।

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score, cohen_kappa_score

# উদাহরণ: যদি validation points থাকে
val_points = gpd.read_file('data/validation_points.shp')

# Raster values extract
val_values = []
for point in val_points.geometry:
    vals = [r.sample([(point.x, point.y)]).__next__()[0] for r in rasters]
    val_values.append(vals)

X_val = pd.DataFrame(val_values, columns=['band1','band2','band3'])
y_val = val_points['class_label']

# Prediction
y_pred_val = rf.predict(X_val)

# Metrics
conf_matrix = confusion_matrix(y_val, y_pred_val)
accuracy = accuracy_score(y_val, y_pred_val)
kappa = cohen_kappa_score(y_val, y_pred_val)

print("Confusion Matrix:\n", conf_matrix)
print(f"Overall Accuracy: {accuracy:.2f}")
print(f"Kappa Coefficient: {kappa:.2f}")

## 6. Visualization of Classified Map

Classification ফলাফল map আকারে দেখানো।

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10,8))
plt.imshow(pred_map, cmap='tab20')
plt.colorbar(label='Classes')
plt.title('Classified Map (Random Forest)')
plt.show()

**পরের পর্বে আমরা জানবো:**  
Accuracy Assessment & Reporting: Confusion Matrix, Overall Accuracy, Kappa Coefficient এবং class-wise accuracy বিশ্লেষণ।